In [1]:
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [2]:
strategy = tf.distribute.TPUStrategy(resolver)

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import joblib

%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
train_df = pd.read_excel("/content/drive/MyDrive/2차입찰금액 예측모델/dataset/2002_2021 최종.xlsx")
test_df = pd.read_excel("/content/drive/MyDrive/2차입찰금액 예측모델/dataset/test.xlsx")




In [ ]:
train_df.columns

Index(['기초금액', '추정가격', '예정가격', '낙차하한율', '사정율', '예가범위', '입찰금액'], dtype='object')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
train_df=train_df.drop(['예정가격','사정율'],axis=1)

In [8]:
train_df.columns

Index(['기초금액', '추정가격', '낙차하한율', '예가범위', '입찰금액'], dtype='object')

In [ ]:
X = train_df.iloc[:, :-1].to_numpy()
y = train_df.iloc[:, -1].to_numpy()
X_test = test_df.to_numpy()

In [9]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 22.7 MB/s eta 0:00:00


In [10]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.9/211.9 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 9.6 MB/s eta 0:00:00


In [11]:
from sklearn.model_selection import train_test_split
X= train_df[['기초금액', '추정가격', '낙차하한율','예가범위']]
y = train_df[['입찰금액']]

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostRegressor,  HistGradientBoostingRegressor, StackingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor

import xgboost as xgb

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler

# optuna.logging.set_verbosity(optuna.logging.WARNING)

# def train(model):
#     X_train, X_test, y_train, y_test = train_test_split(X, y.flatten(), test_size=0.1, random_state=156)
#     y_train = y_train.reshape(-1, 1)
#     y_test  = y_test.reshape(-1, 1)
        
#     model = model.fit(X_train, y_train, early_stopping_rounds=100, verbose=False, eval_set=[(X_test, y_test)])
#     score = mean_squared_error(model.predict(X_train), y_train, squared=False)
#     print(score)
#     return model

/usr/local/lib/python3.8/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


#첫번째시도

In [15]:
param = {
        "n_estimators" : 3348,
        'max_depth':14,
        'min_child_weight':2,
        'gamma':2,
        'learning_rate': 0.01,
        'colsample_bytree':0.9,
        'nthread' : -1,
        'lambda':0.1323097319310655,
        'alpha':0.002599967268252914,
        'subsample': 0.6,
        'random_state': 42
    }
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    

xgb_model = XGBRegressor(**param)
xgb_model.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], early_stopping_rounds = 8, verbose = 1000, eval_metric = 'mae')

score = mean_squared_error(xgb_model.predict(X_test), y_test, squared=False)

[0]	validation_0-mae:170051018.34558	validation_1-mae:168259529.32393
[878]	validation_0-mae:925449.69540	validation_1-mae:1110225.10054


In [16]:
print(score)
print(xgb_model.score(X_train,y_train))
print(xgb_model.score(X_test,y_test))

6075624.110516524
0.9999144629006973
0.9998506883264067


In [ ]:
print(score)
print(xgb_model.score(X_train,y_train))
print(xgb_model.score(X_test,y_test))

9238956.576733997
0.9999544651023425
0.9997443942646542


#두번째 시도

In [17]:
param = {
        "n_estimators" : 2728,
        'max_depth':14,
        'min_child_weight':2,
        'gamma':2,
        'learning_rate': 0.01,
        'colsample_bytree':0.9,
        'nthread' : -1,
        'lambda':0.07578679235936187,
        'alpha':0.001453153277977882,
        'subsample': 0.6,
        'random_state': 42
    }
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    

model = XGBRegressor(**param)
model.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], early_stopping_rounds = 8, verbose = 1000, eval_metric = 'mae')

score = mean_squared_error(xgb_model.predict(X_test), y_test, squared=False)

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mae:169777655.74431	validation_1-mae:170720567.82823
[867]	validation_0-mae:915594.69306	validation_1-mae:1152477.10685


In [18]:
print(score)
print(xgb_model.score(X_train,y_train))
print(xgb_model.score(X_test,y_test))

5587858.623146966
0.9999104306197691
0.9998990913459455


In [ ]:
print(score)
print(xgb_model.score(X_train,y_train))
print(xgb_model.score(X_test,y_test))

# 세번째 시도

In [35]:
param = {
        "n_estimators" : 1357,
        'max_depth':7,
        'min_child_weight':2,
        'gamma':1,
        'learning_rate': 0.01,
        'colsample_bytree':0.8,
        'nthread' : -1,
        'lambda': 0.11828779860456479,
        'alpha':0.024080811696866,
        'subsample': 0.6,
        'random_state': 42
    }

    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    

model_2 = XGBRegressor(**param)
model_2.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], early_stopping_rounds = 8, verbose = 1000, eval_metric = 'mae')

score = mean_squared_error(xgb_model.predict(X_test), y_test, squared=False)

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mae:170359762.02260	validation_1-mae:165484003.44206
[838]	validation_0-mae:1093088.99400	validation_1-mae:1126318.69200


In [36]:
print(score)
print(model_2.score(X_train,y_train))
print(model_2.score(X_test,y_test))

3136594.0505034393
0.9998809447230503
0.9998902900753592


In [34]:
print(score)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

5510699.504681531
0.9998905943734013
0.9997637626503899


In [30]:
from keras.models import load_model

model.save_model("xgb_model_3_6.json")

In [31]:
model_4 = XGBRegressor()

model_4.load_model("/content/model_sklearn_10.json")

In [32]:
print(score)
print(model_4.score(X_train,y_train))
print(model_4.score(X_test,y_test))

3992286.7720864927
0.999871888616719
0.9998977949082868


#세번째 시도

#네번째 시도

In [37]:
param = {
        
        "n_estimators" : 3348,
        'max_depth':14,
        'min_child_weight':2,
        'gamma':2,
        'learning_rate': 0.01,
        'colsample_bytree':0.9,
        'nthread' : -1,
        'lambda':  0.1323097319310655,
        'alpha':0.002599967268252914,
        'subsample' : 0.6
    }
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    

model = XGBRegressor(**param)
model.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], early_stopping_rounds = 8, verbose = 1000, eval_metric = 'mae')

score = mean_squared_error(xgb_model.predict(X_test), y_test, squared=False)

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mae:169464812.09318	validation_1-mae:173525313.85562
[794]	validation_0-mae:942911.29857	validation_1-mae:1237732.02822


In [38]:
print(score)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

4517713.450992718
0.9998979774960258
0.9995626646102237


In [ ]:
print(score)
print(xgb_model.score(X_train,y_train))
print(xgb_model.score(X_test,y_test))

4050406.6337061957
0.9999003969216668
0.999942927499448


In [ ]:
print(score)
print(xgb_model.score(X_train,y_train))
print(xgb_model.score(X_test,y_test))

4387534.430525755
0.9999001533133395
0.9999403363675868


In [ ]:
from keras.models import load_model

xgb_model.save_model("model_sklearn_2.json")

In [ ]:
model3 = XGBRegressor()

model3.load_model("/content/drive/MyDrive/model_sklearn_2.json")

In [ ]:
print(model3.score(X_train,y_train))
print(model3.score(X_test,y_test))

0.9999001533133395
0.9999403363675868


In [ ]:
print(model3.score(X_train,y_train))
print(model3.score(X_test,y_test))

0.9999001533133395
0.9999403363675868


In [ ]:
test_df = pd.read_excel("/content/drive/MyDrive/2차입찰금액 예측모델/dataset/2022.xlsx")

In [ ]:
from sklearn.model_selection import train_test_split
a= test_df[['기초금액', '추정가격', '낙차하한율','예가범위']]
b = train_df[['입찰금액']]

In [ ]:
from keras.models import load_model

model.save_model("model_sklearn.json")

In [ ]:
model3 = XGBRegressor()

model3.load_model("/content/drive/MyDrive/model_sklearn_2.json")

In [ ]:
print(model3.score(X_train,y_train))
print(model3.score(X_test,y_test))

0.9999001533133395
0.9999403363675868


In [ ]:
y_pred = model3.predict(X_test)
print(y_pred[:50])

[3.29073880e+07 2.66735660e+07 5.44751040e+08 4.35335800e+07
 6.89157680e+07 1.04584992e+08 3.03492838e+09 7.18969360e+07
 2.29267020e+07 3.40979872e+08 3.48176920e+07 4.84770800e+07
 1.11714610e+07 1.25217480e+08 1.25507240e+08 7.11445120e+07
 6.83330560e+07 2.39734980e+07 4.87793200e+07 1.31089336e+08
 2.26969140e+07 1.22704208e+08 4.61523360e+08 8.70738720e+07
 5.51117960e+07 5.32814680e+07 5.66109320e+07 9.66196160e+07
 6.29480640e+07 7.49084320e+07 4.35335800e+07 7.23557040e+07
 4.04045480e+07 5.23055880e+07 1.92123149e+09 3.72615440e+07
 4.30208600e+07 2.89080736e+08 7.11445120e+07 4.16719600e+07
 1.30589632e+09 2.73014980e+07 6.00741056e+08 3.57047640e+07
 2.51719500e+07 8.54468480e+07 1.21272504e+08 2.18770860e+07
 3.63524352e+08 4.64189160e+07]


In [ ]:
y_test[:50]

,입찰금액
34799,3.323835e+07
510712,2.665476e+07
12616,5.484849e+08
443611,4.385495e+07
458052,6.856866e+07
514479,1.041705e+08
389625,3.051656e+09
65344,7.247873e+07
408230,2.258999e+07
329144,3.430380e+08


In [ ]:
y_pred = model3.predict(a)

In [ ]:
y_pred[:50]

array([5.40533920e+07, 5.68667520e+07, 1.22167376e+08, 4.01616736e+08,
       7.70880512e+08, 1.60819648e+08, 2.57311584e+08, 6.37130560e+07,
       1.48034944e+08, 8.47072720e+07, 3.04651880e+07, 1.23398856e+08,
       2.54992320e+07, 8.51132096e+08, 5.60347080e+07, 6.42492520e+07,
       1.39882944e+08, 3.48729280e+07, 3.73504280e+07, 2.66735660e+07,
       1.82880380e+07, 1.85612100e+07, 3.55667240e+07, 1.14925120e+08,
       2.45824660e+07, 7.95903040e+07, 6.94061760e+07, 3.91244920e+07,
       2.81764608e+08, 1.33625824e+08, 6.18205000e+07, 1.73774608e+08,
       1.12857776e+08, 7.06482752e+08, 1.46497952e+08, 8.66906480e+07,
       4.48550680e+07, 4.78400640e+07, 1.23333464e+08, 6.22113960e+07,
       5.74659800e+07, 2.89568900e+07, 3.37841728e+08, 5.15989760e+07,
       1.85916256e+08, 8.96923392e+08, 9.90310080e+07, 5.78490800e+07,
       2.74318310e+09, 1.86188304e+08], dtype=float32)

In [ ]:
y_pred[:50]

array([5.40533920e+07, 5.68667520e+07, 1.22167376e+08, 4.01616736e+08,
       7.70880512e+08, 1.60819648e+08, 2.57311584e+08, 6.37130560e+07,
       1.48034944e+08, 8.47072720e+07, 3.04651880e+07, 1.23398856e+08,
       2.54992320e+07, 8.51132096e+08, 5.60347080e+07, 6.42492520e+07,
       1.39882944e+08, 3.48729280e+07, 3.73504280e+07, 2.66735660e+07,
       1.82880380e+07, 1.85612100e+07, 3.55667240e+07, 1.14925120e+08,
       2.45824660e+07, 7.95903040e+07, 6.94061760e+07, 3.91244920e+07,
       2.81764608e+08, 1.33625824e+08, 6.18205000e+07, 1.73774608e+08,
       1.12857776e+08, 7.06482752e+08, 1.46497952e+08, 8.66906480e+07,
       4.48550680e+07, 4.78400640e+07, 1.23333464e+08, 6.22113960e+07,
       5.74659800e+07, 2.89568900e+07, 3.37841728e+08, 5.15989760e+07,
       1.85916256e+08, 8.96923392e+08, 9.90310080e+07, 5.78490800e+07,
       2.74318310e+09, 1.86188304e+08], dtype=float32)

In [ ]:
dPy_pred[1]

26673566.0

In [ ]:
y_pred[2]

544751040.0

In [ ]:
y_test

,입찰금액
199237,8.107095e+08
84168,3.323436e+07
456379,3.544174e+07
128141,8.735370e+07
47180,4.163062e+07
...,...
178461,2.638350e+07
56111,2.083599e+07
152911,4.558959e+07
247792,3.057736e+07


In [ ]:
y_pred[:50]

array([5.4063660e+07, 5.6940236e+07, 1.2230476e+08, 4.0245686e+08,
       7.7442278e+08, 1.6048774e+08, 2.5745418e+08, 6.3718320e+07,
       1.4821144e+08, 8.4782456e+07, 3.0086146e+07, 1.2340924e+08,
       2.5503256e+07, 8.5093997e+08, 5.6024828e+07, 6.4191632e+07,
       1.3979526e+08, 3.4886500e+07, 3.7316056e+07, 2.6645862e+07,
       1.8226832e+07, 1.8722210e+07, 3.5482108e+07, 1.1486289e+08,
       2.4499892e+07, 7.9252688e+07, 6.9439312e+07, 3.9047636e+07,
       2.8200944e+08, 1.3383016e+08, 6.1458252e+07, 1.7234416e+08,
       1.1272485e+08, 7.0658586e+08, 1.4653733e+08, 8.6264176e+07,
       4.4805252e+07, 4.7825080e+07, 1.2325374e+08, 6.2236516e+07,
       5.7457208e+07, 2.8901548e+07, 3.3715139e+08, 5.1630556e+07,
       1.8594650e+08, 8.9841152e+08, 9.9017208e+07, 5.7785444e+07,
       2.7390845e+09, 1.8654710e+08], dtype=float32)

#다섯번째 시도

In [39]:
param = {
        "n_estimators" : 5000,
        'max_depth':7,
        'min_child_weight':1,
        'gamma':2,
        'learning_rate': 0.01,
        'colsample_bytree':0.8,
        'nthread' : -1,
        'lambda':  0.329047999343297,
        'alpha':0.0028612856514716894,
        'subsample': 0.6,
        'random_state': 42
    }
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    

model = XGBRegressor(**param)
model.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], early_stopping_rounds = 8, verbose = 1000, eval_metric = 'mae')

score = mean_squared_error(xgb_model.predict(X_test), y_test, squared=False)

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mae:169904809.98520	validation_1-mae:169589861.84888
[920]	validation_0-mae:1069958.35686	validation_1-mae:1139691.38965


In [40]:
print(score)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

3722997.591556649
0.9999124422991031
0.9999078585267509


In [ ]:
print(score)
print(xgb_model.score(X_train,y_train))
print(xgb_model.score(X_test,y_test))

4419411.283136086
0.9999028105185006
0.9999222534948741


In [ ]:
test_df = pd.read_excel("/content/drive/MyDrive/2차입찰금액 예측모델/dataset/2022.xlsx")

In [ ]:
train_df=train_df.drop(['예정가격','사정율'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
a= test_df[['기초금액', '추정가격', '낙차하한율','예가범위']]
b = train_df[['입찰금액']]

In [ ]:
y_pred = xgb_model.predict(a)

In [ ]:
y_pred[:50]

array([5.40533920e+07, 5.68667520e+07, 1.22167376e+08, 4.01616736e+08,
       7.70880512e+08, 1.60819648e+08, 2.57311584e+08, 6.37130560e+07,
       1.48034944e+08, 8.47072720e+07, 3.04651880e+07, 1.23398856e+08,
       2.54992320e+07, 8.51132096e+08, 5.60347080e+07, 6.42492520e+07,
       1.39882944e+08, 3.48729280e+07, 3.73504280e+07, 2.66735660e+07,
       1.82880380e+07, 1.85612100e+07, 3.55667240e+07, 1.14925120e+08,
       2.45824660e+07, 7.95903040e+07, 6.94061760e+07, 3.91244920e+07,
       2.81764608e+08, 1.33625824e+08, 6.18205000e+07, 1.73774608e+08,
       1.12857776e+08, 7.06482752e+08, 1.46497952e+08, 8.66906480e+07,
       4.48550680e+07, 4.78400640e+07, 1.23333464e+08, 6.22113960e+07,
       5.74659800e+07, 2.89568900e+07, 3.37841728e+08, 5.15989760e+07,
       1.85916256e+08, 8.96923392e+08, 9.90310080e+07, 5.78490800e+07,
       2.74318310e+09, 1.86188304e+08], dtype=float32)

#6번째시도

In [ ]:
param = {
        "n_estimators" : 1732,
        'max_depth':9,
        'min_child_weight':2,
        'gamma':1,
        'learning_rate': 0.01,
        'colsample_bytree':0.7,
        'nthread' : -1,
        'lambda':  0.17594274400524043,
        'alpha':0.0036027077351499267,
        'subsample': 0.6,
        'random_state': 42
    }
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    

model = XGBRegressor(**param)
model.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], early_stopping_rounds = 8, verbose = 1000, eval_metric = 'mae')

score = mean_squared_error(xgb_model.predict(X_test), y_test, squared=False)

[0]	validation_0-mae:169632034.20816	validation_1-mae:172136012.52917


/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[1000]	validation_0-mae:1383915.29435	validation_1-mae:1614312.69448
[1613]	validation_0-mae:1232059.90137	validation_1-mae:1459337.34849


In [ ]:
print(score)
print(xgb_model.score(X_train,y_train))
print(xgb_model.score(X_test,y_test))

5877226.657012626
0.9999057385773407
0.9998933484993054


In [ ]:
print(score)
print(xgb_model.score(X_train,y_train))
print(xgb_model.score(X_test,y_test))

4593380.990955596
0.9999009631169379
0.9999338765042348


#7번째시도

In [ ]:
param = {
        "n_estimators" : 2230,
        'max_depth':12,
        'min_child_weight':2,
        'gamma':2,
        'learning_rate': 0.01,
        'colsample_bytree':0.8,
        'nthread' : -1,
        'lambda':  0.18791192524144085,
        'alpha':0.0022251798086885414,
        'subsample': 0.6,
        'random_state': 42
    }
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    

model = XGBRegressor(**param)
model.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], early_stopping_rounds = 8, verbose = 1000, eval_metric = 'mae')

score = mean_squared_error(xgb_model.predict(X_test), y_test, squared=False)

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mae:170217627.13585	validation_1-mae:166763759.41770
[798]	validation_0-mae:973581.80394	validation_1-mae:1167366.54102


In [ ]:
print(score)
print(xgb_model.score(X_train,y_train))
print(xgb_model.score(X_test,y_test))

4638445.854764215
0.9999029731690857
0.9999194254700882


In [ ]:
print(score)
print(xgb_model.score(X_train,y_train))
print(xgb_model.score(X_test,y_test))

34799398.11138971
0.999690950074633
0.9965542572013315


#8번째시도

In [ ]:
param = {
        "n_estimators" : 1357,
        'max_depth':7,
        'min_child_weight':2,
        'gamma':1,
        'learning_rate': 0.01,
        'colsample_bytree':0.8,
        'nthread' : -1,
        'lambda':0.11828779860456479,
        'alpha':0.024080811696866,
        'subsample': 0.6,
        'random_state': 42
    }

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    

model = XGBRegressor(**param)
model.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], early_stopping_rounds = 8, verbose = 1000, eval_metric = 'mae')

score = mean_squared_error(xgb_model.predict(X_test), y_test, squared=False)

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mae:170031496.20320	validation_1-mae:168433241.87975
[805]	validation_0-mae:1088489.34111	validation_1-mae:1178756.85662


In [ ]:
print(score)
print(xgb_model.score(X_train,y_train))
print(xgb_model.score(X_test,y_test))

6295712.42102359
0.9999086908201787
0.9998641928417055


In [ ]:
test_df = pd.read_excel("/content/drive/MyDrive/2차입찰금액 예측모델/dataset/2022.xlsx")

In [ ]:
from sklearn.model_selection import train_test_split
a= test_df[['기초금액', '추정가격', '낙차하한율','예가범위']]
b = train_df[['입찰금액']]

In [ ]:
y_pred = xgb_model.predict(a)

In [ ]:
y_pred[:50]

- - - - 

In [ ]:
xgb = XGBRegressor(**xgb_param2)
xgb.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], early_stopping_rounds = 8, verbose = 1000, eval_metric = 'mae')

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mae:169470096.42180	validation_1-mae:173487751.11845
[1000]	validation_0-mae:983082.39591	validation_1-mae:1119802.33633
[1247]	validation_0-mae:976887.68425	validation_1-mae:1114901.61668


XGBRegressor(alpha=0, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=1, learning_rate=0.01,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=1.0, missing=nan, monotone_constraints=None,
             n_estimators=10000, n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
print(xgb.score(X_train,y_train))
print(xgb.score(X_test,y_test))

0.9999237527154113
0.9998276916575857


In [ ]:
param = {
        "n_estimators" : 3348,
        'max_depth':14,
        'min_child_weight':2,
        'gamma':2,
        'learning_rate': 0.01,
        'colsample_bytree':0.9,
        'nthread' : -1,
        'lambda':   0.1323097319310655,
        'alpha': 0.002599967268252914,
        'subsample': 0.6,
        'random_state': 42
    }
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    
y_train = y_train.values.reshape(-1, 1)
y_test  = y_test.values.reshape(-1, 1)

model = XGBRegressor(**param)
xgb_model = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])
score = mean_squared_error(xgb_model.predict(X_test), y_test, squared=False)

In [ ]:
print(score)
print(xgb_model.score(X_train,y_train))
print(xgb_model.score(X_test,y_test))

9279856.148274241
0.9999488464593171
0.9997189216760661


In [ ]:
def objectiveXGB(trial: Trial, X, y, test):
    param = {
        "n_estimators" : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth':trial.suggest_int('max_depth', 4, 16),
        'min_child_weight':trial.suggest_int('min_child_weight', 1, 300),
        'gamma':trial.suggest_int('gamma', 1, 3),
        'learning_rate': 0.01,
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.1),
        'nthread' : -1,
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
        'random_state': 42
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    
    y_train = y_train.values.reshape(-1, 1)
    y_test  = y_test.values.reshape(-1, 1)

    model = xgb.XGBRegressor(**param)
    xgb_model = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])
    score = mean_squared_error(xgb_model.predict(X_test), y_test, squared=False)

    return score

In [ ]:
study = optuna.create_study(direction='minimize',sampler=TPESampler())
study.optimize(lambda trial : objectiveXGB(trial, X,  y, X_test), n_trials=50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

# best_param = study.best_trial.params
# xgbReg = train(xgb.XGBRegressor(**best_param, tree_method='gpu_hist', random_state=42, predictor='gpu_predictor', learning_rate=0.01, nthread=-1))

#params =  {'n_estimators': 3520, 'max_depth': 11, 'min_child_weight': 231, 'gamma': 2, 'colsample_bytree': 0.7, 'lambda': 0.014950936465569798, 'alpha': 0.28520156840812494, 'subsample': 0.6}
#xgbReg = train(xgb.XGBRegressor(**params, tree_method='gpu_hist', random_state=42, predictor='gpu_predictor', learning_rate=0.01, nthread=-1))

# 0.6744648190960726

[I 2023-03-02 13:34:35,900] A new study created in memory with name: no-name-699d2b34-52de-4edb-8e0a-bdeb9dc323fd
C:\Users\jmsy1\AppData\Local\Temp\ipykernel_21132\531827735.py:8: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.1),
C:\Users\jmsy1\AppData\Local\Temp\ipykernel_21132\531827735.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
C:\Users\jmsy1\AppData\Local\Temp\ipykernel_21132\531827735.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
def objectiveLGBM(trial: Trial, X, y, test):
    param = {
        'objective': 'regression',
        'metric': 'root_mean_squared_error',https://github.com/optuna/optuna/releases/tag/v3.0.0.
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'learning_rate': 0.01,
        'n_estimators': trial.suggest_int('n_estimators', 700, 3000),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'device':"gpu",
        'gpu_use_dp':True
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y.flatten(), test_size=0.1)
        
    lgbm_regr = LGBMRegressor(**param)
    lgbm_regr = lgbm_regr.fit(X_train, y_train, verbose=False)
    
    score = mean_squared_error(lgbm_regr.predict(X_test), y_test, squared=False)
    return score

In [ ]:
study = optuna.create_study(direction='minimize',sampler=TPESampler())
study.optimize(lambda trial : objectiveLGBM(trial, X, y, X_test), n_trials=20)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

best_param2 = study.best_trial.params
lgbm = LGBMRegressor(**best_param2, device="gpu",gpu_use_dp=True, objective='regression', metric='root_mean_squared_error',  learning_rate= 0.01, boosting_type='gbdt')

# Best trial: score 0.6934602592622415,
# params {'lambda_l1': 4.168316306871167e-05, 'lambda_l2': 1.1288557039193647e-05, 'num_leaves': 98, 'n_estimators': 2280, 'feature_fraction': 0.7977209715952681, 'bagging_fraction': 0.4353577523638581, 'bagging_freq': 4, 'min_child_samples': 69}

In [ ]:
from vecstack import stacking

final_model = xgb.XGBRegressor(n_estimators= 2000, max_depth= 16,tree_method='gpu_hist', predictor='gpu_predictor')
sgd = SGDRegressor(max_iter=1000)
hgb = HistGradientBoostingRegressor( max_depth=3, min_samples_leaf=1)
cat = CatBoostRegressor(task_type="GPU", verbose=False)

estimators = [
    lgbm, cat, sgd, hgb, xgbReg
]

S_train, S_test = stacking(estimators,  X, y, X_test, regression=True, metric=mean_squared_error, n_folds=5, 
                           shuffle=False, random_state=0, verbose=2)
final_model.fit(S_train, y)

print(mean_squared_error(final_model.predict(S_train), y, squared=False))

In [ ]:
submission = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/test.csv", index_col=["id"])
y_hat = final_model.predict(S_test)        

submission["target"] = y_hat 
submission[["target"]].to_csv("/kaggle/working/submission_stacking.csv")
joblib.dump(final_model, '/kaggle/working/skacking.pkl')

In [ ]:
submission = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/test.csv", index_col=["id"])
lgbm = LGBMRegressor(**best_param2, device="gpu",gpu_use_dp=True, objective='regression', learning_rate= 0.01, metric='root_mean_squared_error', boosting_type='gbdt')
lgbm = lgbm.fit(X, y, verbose=False)
y_hat = lgbm.predict(submission.to_numpy())        
print(mean_squared_error(lgbm.predict(X), y, squared=False))
submission["target"] = y_hat 
submission[["target"]].to_csv("/kaggle/working/submission_lgbm.csv")
joblib.dump(lgbm, '/kaggle/working/lgbm.pkl')

In [ ]:
submission = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/test.csv", index_col=["id"])
params =  {'n_estimators': 3520, 'max_depth': 11, 'min_child_weight': 231, 'gamma': 2, 'colsample_bytree': 0.7, 'lambda': 0.014950936465569798, 'alpha': 0.28520156840812494, 'subsample': 0.6}
xgbReg = train(xgb.XGBRegressor(**params, tree_method='gpu_hist', random_state=42, predictor='gpu_predictor', learning_rate=0.01, nthread=-1))

y_hat = xgbReg.predict(submission.to_numpy())        
print(mean_squared_error(xgbReg.predict(X), y, squared=False))

submission["target"] = y_hat 
submission[["target"]].to_csv("/kaggle/working/submission_xgb.csv")
joblib.dump(xgbReg, '/kaggle/working/xgb_reg.pkl')